Futmondo no tiene API así que hay que hacer un par de cosas antes:
 - Ir a MERCADO
 - Cargar todas las noticias hasta el día que se quiera
 - Inspeccionar elemento, buscar `pressReleases`
 - Edit as HTML, Select all, Copy, Paste en `futmondo.txt`

In [1]:
import pandas as pd

def transform_data(transfer):
    transfer = transfer.split('class="text"><strong>')[1]
    transfer = transfer.split('</strong></div></li><a class="icon-revert"></a></ul>')[0]
    transfer = transfer.replace(
        '</strong><time>', ' - ').replace(
        '</time><span>', ' - ').replace(
        '</span><div class="from">De <strong>', ' - ').replace(
        '</strong> a <strong>', ' - ')

    # At this point every transfer should look like: 
    # Antoñín - 17/09/2020 - 10:10 - 800.000 € - Equipo ESPAÑA - futmondo
    
    transfer_data = transfer.split(' - ')
    transfer_dict = {
        'player': transfer_data[0],
        'date': transfer_data[1],
        'time': transfer_data[2],
        'amount': transfer_data[3].replace('.', '').replace(' €', ''),
        'from': transfer_data[4],
        'to': transfer_data[5],
    }
    return transfer_dict

In [2]:
with open('futmondo.txt', 'r', encoding="utf8") as f:
    transfers = f.readlines()[0].split('<li class="post pressRelease">')
    transfers = [x.strip() for x in transfers]
    transfers = transfers[1:]

transfers = [transform_data(t) for t in transfers]

df = pd.DataFrame(transfers)
df.drop('time', axis=1, inplace=True)
df['date'] =  pd.to_datetime(df['date'], format='%d/%m/%Y')
df['amount'] =  pd.to_numeric(df['amount'])

In [3]:
df_agg = df.groupby(['date', 'from', 'to'], as_index=False).agg(
        {'amount': 'sum', 
         'player': lambda x: ', '.join(x)})

In [4]:
miguel = df_agg[(df_agg['from'] == 'futmondo') & (df_agg['to'] == 'El Dream Team')]

In [57]:
miguel.set_index(miguel.date, inplace=True)
miguel = miguel.resample('D').sum()#.fillna(0)
miguel.head(25)

,amount
date,
2020-08-13,5701000
2020-08-14,58400000
2020-08-15,28500000
2020-08-16,18699999
2020-08-17,60400000
2020-08-18,37899999
2020-08-19,0
2020-08-20,54600000
2020-08-21,12000000


In [40]:
idx = pd.period_range(df['date'].min(), df['date'].max())

miguel['date'].index = pd.DatetimeIndex(miguel['date'].index)
miguel['date'] = miguel['date'].reindex(idx, fill_value=0)
miguel.head(25)

C:\Users\ruben\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,date,from,to,amount,player
0,NaN,futmondo,El Dream Team,5701000,"Miramón, Nacho Vidal"
5,NaN,futmondo,El Dream Team,58400000,"Óliver Torres, Álex Fdez, Jesús Navas"
9,NaN,futmondo,El Dream Team,28500000,Aitor Fernández
14,NaN,futmondo,El Dream Team,18699999,"Kike García, Koke Vegas, Víctor Díaz"
20,NaN,futmondo,El Dream Team,60400000,"Joaquín, Marcos Llorente, Lucas Pérez"
25,NaN,futmondo,El Dream Team,37899999,"Roberto Torres, Darwin Machís"
36,NaN,futmondo,El Dream Team,54600000,"Kounde, Vinícius, Campaña"
41,NaN,futmondo,El Dream Team,12000000,Coquelin
46,NaN,futmondo,El Dream Team,29550000,"Ansu Fati, Trippier"
50,NaN,futmondo,El Dream Team,35880000,"Enes Ünal, Domingos Duarte, Luuk de Jong"
